In [33]:
import cPickle as pickle
import gzip
import numpy
from midi_to_statematrix import *

import multi_training
import model
import RBM

def gen_adaptive(m, pcs, times, keep_thoughts=False, name="final", rbm = False):
    xIpt, xOpt = map(lambda x: numpy.array(x, dtype='int8'), multi_training.getPieceSegment(pcs))
    all_outputs = [xOpt[0]]
    if keep_thoughts:
        all_thoughts = []
    m.start_slow_walk(xIpt[0])
    cons = 1
    for time in range(multi_training.batch_len * times):
        resdata = m.slow_walk_fun(cons)
        nnotes = numpy.sum(resdata[-1][:, 0])
        if nnotes < 2:
            if cons > 1:
                cons = 1
            cons -= 0.02
        else:
            cons += (1 - cons) * 0.3
        all_outputs.append(resdata[-1])
        if keep_thoughts:
            all_thoughts.append(resdata)
    if rbm:
        print numpy.array(all_outputs).shape
        rbm_outputs = rbm_reconstruct(pcs, numpy.array(all_outputs))
        noteStateMatrixToMidi(numpy.array(rbm_outputs), 'output/' + name + ' rbm')
    noteStateMatrixToMidi(numpy.array(all_outputs), 'output/' + name)
    if keep_thoughts:
        pickle.dump(all_thoughts, open('output/' + name + '.p', 'wb'))

In [ ]:
#pieces_major = multi_training.loadPieces('music/major')
#pieces_minor = multi_training.loadPieces('music/minor')
path = 'C_music'
pieces = multi_training.loadPieces(dirpath = path)

In [2]:
m = model.Model([300, 300], [100, 50], dropout=0.5)

model.py:367: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  self.walk_input: next_input
model.py:370: UserWarning: Updating an `OrderedUpdates` with a non-ordered dictionary with 2+ elements could make your code non-deterministic
  updates.update({hidden:newstate for hidden, newstate, layer in zip(self.walk_hiddens, new_states, self.time_model.layers) if has_hidden(layer)})


In [6]:
m.learned_config = pickle.load(open("output/final_learned_config_7000.p", "rb"))

In [29]:
batches = 7000
batches_old = 0

print 'Training {0}+{1} batches on {2}'.format(batches,batches_old, path)

multi_training.trainPiece(m, pieces, [batches, batches_old])#, notes_to_input = None)
# add starting output names from given batch
pickle.dump(m.learned_config, open("output/final_learned_config_{0}.p".format(1*batches), "wb"))

Training 7000+0 batches on C_music


In [34]:
gen_adaptive(m,pieces,1,name="composition_{0}".format(batches), rbm=True)

(129, 78, 2)


NameError: global name 'rbm_reconstruct' is not defined

In [16]:
import numpy as np
a = np.arange(253*2002*78*2).reshape((253,2002,78,2))
a = a.reshape(-1, 16*78*2)
a.shape

ValueError: total size of new array must be unchanged

In [37]:
255*2111*78*2%(16*78*2.0)

156.0